In [ ]:
import pandas as pd
import numpy as np
import ebaysdk
import gspread
import requests
import time
import numpy as np
from scipy import stats
import csv
import os
from dotenv import load_dotenv

In [ ]:
# connect to google sheets and test connection

load_dotenv()
creds_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
gc = gspread.service_account(filename=creds_path)

spreadsheet_id = os.getenv("SPREADSHEET_ID")
sh = gc.open_by_key(spreadsheet_id)

worksheet_list = sh.worksheets()
print(f"Connection successful. Found {len(worksheet_list)} sheets.")

In [ ]:
def build_query(row):
    # 1. Essential Card Data
    year = str(row.get('display_year', row.get('year', '')))
    
    
    components = [
        year,
        str(row.get('manufacturer', '')),
        
        str(row.get('product_name', '')), 
        str(row.get('player_fname', '')),
        str(row.get('player_lname', '')),
        str(row.get('card_number', '')), # Your existing card number
        str(row.get('parallel_color', '')),
        str(row.get('variation_name', ''))
    ]

    
    if str(row.get('rookie', '')).upper() == 'TRUE':
        components.append("RC")
    if str(row.get('autograph', '')).upper() == 'TRUE':
        components.append("AUTO")

    
    final_parts = []
    for c in components:
        val = str(c).strip()
        if val and val.upper() not in ["FALSE", "NONE"] and val not in final_parts:
            final_parts.append(val)
            
    return " ".join(final_parts)

In [ ]:
# Initialize csv file for search failures

with open('failed_searches.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Sheet Name", "Row Index", "Query", "Error Type"])

# 1. Configuration
load_dotenv()
EBAY_TOKEN = os.getenv("EBAY_TOKEN")
SPREADSHEET_ID = os.getenv("SPREADSHEET_ID")
target_col_name = "average_price"


def get_ebay_avg(query, sheet_name, row_idx):
    url = "https://api.ebay.com/buy/browse/v1/item_summary/search"
    headers = {
        "Authorization": f"Bearer {EBAY_TOKEN}",
        "X-EBAY-C-MARKETPLACE-ID": "EBAY_US"
    }
    params = {
        "q": query + " -psa -bgs -sgc -cgc -graded -slab -pca",
        "filter": "buyingOptions:{FIXED_PRICE|AUCTION},lastSoldDate:[2025-12-01T00:00:00Z..2026-01-31T00:00:00Z],conditionIds:{4000}",
        "limit": 100
    }
    try:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            items = response.json().get('itemSummaries', [])
            
            
            if not items:
                log_failure(sheet_name, row_idx, query, "No Results Found")
                return 0
                
            prices = [float(i['price']['value']) for i in items]
            if len(prices) > 0:
                if len(prices) < 5:
                    return round(np.median(prices), 2)
                else:
                    z_scores = np.abs(stats.zscore(prices))
                    filtered = np.array(prices)[z_scores < 2]
                    return round(np.mean(filtered), 2)
            return 0
        else:
           
            log_failure(sheet_name, row_idx, query, f"API Error {response.status_code}")
            return 0
    except Exception as e:
        
        log_failure(sheet_name, row_idx, query, str(e))
        return 0

def log_failure(sheet_name, row_idx, query, error_msg):
    with open('failed_searches.csv', mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([sheet_name, row_idx, query, error_msg])

# 2. Main Processing Loop
spreadsheet = gc.open_by_key(SPREADSHEET_ID)
all_sheets = spreadsheet.worksheets()

for ws in all_sheets:
    print(f"\n--- Processing Sheet: {ws.title} ---")
    data = ws.get_all_records()
    if not data:
        print(f"Skipping {ws.title}: No data found.")
        continue
    
    df = pd.DataFrame(data)
    
   
    if target_col_name not in df.columns:
        print(f"Adding '{target_col_name}' column to {ws.title}...")
        new_col_idx = len(df.columns) + 1
        ws.update_cell(1, new_col_idx, target_col_name)
        df[target_col_name] = 0
        col_idx = new_col_idx
    else:
        col_idx = df.columns.get_loc(target_col_name) + 1

    # 3. Update Prices 
    
    for index, row in df.iterrows():
        query = build_query(row) 
        
        
        avg_price = get_ebay_avg(query, ws.title, index + 2)
        
        # Write back to Google Sheet
        ws.update_cell(index + 2, col_idx, avg_price)
        print(f"Updated {ws.title} Row {index + 2}: {query} -> ${avg_price}")
        
        
        time.sleep(1.5)

print("\nAll 13 sheets processed!")